# Notebook Summary

The purpose of this notebook is to implement a trading strategy where you're buying/selling dogecoin based on sentiment from Elon Musk's twitter feed 


# Use a .env File to Store your API Credentials and Connect To APIs

##  Alpaca

Make sure you have a .env file set up with your Alpaca API keys. The notebook is looking for your Alpaca API keys in this format: 

API_KEY = "xxx"

API_SECRET = "xxx"

## Twitter

Link to Twitter Developer API Portal: https://developer.twitter.com/en

Make sure you also store your credentials for the twitter developer api in your .env file. 
This notebook assumes that you have "essential" access (please note that you can request a higher access level)

The additional Twitter credentials you need to store in the .env file are the following: 

TWITTER_API_KEY = "XXX"

TWITTER_API_KEY = "XXX"

TWITTER_API_SECRET_KEY = "XXX"

TWITTER_BEARER_TOKEN = "XXX"

TWITTER_ACCESS_TOKEN = "XXX"

TWITTER_ACCESS_TOKEN_SECRET = "XXX"

In [1]:
import pandas as pd
from twarc import Twarc2, expansions
import json
from dotenv import load_dotenv
import os
import alpaca_trade_api as tradeapi
from alpaca_trade_api.rest import TimeFrame, TimeFrameUnit
import datetime
import tweepy
import requests
from dotenv import load_dotenv
from datetime import datetime
import pandas as pd
import nltk
from nltk.sentiment.vader import SentimentIntensityAnalyzer
import nltk
nltk.download('vader_lexicon')

#Twitter API Github Examples Found Here:
## https://github.com/twitterdev/getting-started-with-the-twitter-api-v2-for-academic-research/blob/main/modules/6a-labs-code-academic-python.md

[nltk_data] Downloading package vader_lexicon to
[nltk_data]     C:\Users\PAULA\AppData\Roaming\nltk_data...
[nltk_data]   Package vader_lexicon is already up-to-date!


True

In [2]:
# Load the environment variables from the .env file
load_dotenv()

True

# Connect to Alpaca Paper Trading API

In [3]:
API_KEY = os.getenv("ALPACA_API_KEY")
API_SECRET = os.getenv("ALPACA_SECRET_KEY")
ALPACA_API_BASE_URL = "https://paper-api.alpaca.markets" #notice "PAPER" in the URL

In [4]:
# Create a connection to the API 
api = tradeapi.REST(API_KEY, API_SECRET, ALPACA_API_BASE_URL, api_version="v2")

# Create client for Twitter API

In [5]:
# grab twitter keys, following instructions from github: 

#client = Twarc2(bearer_token=os.getenv("TWITTER_BEARER_TOKEN"))
TWITTER_API_KEY = os.getenv("TWITTER_API_KEY")
TWITTER_API_SECRET_KEY = os.getenv("TWITTER_API_SECRET_KEY")
TWITTER_BEARER_TOKEN = os.getenv("TWITTER_BEARER_TOKEN")
TWITTER_ACCESS_TOKEN = os.getenv("TWITTER_ACCESS_TOKEN")
TWITTER_ACCESS_TOKEN_SECRET = os.getenv("TWITTER_ACCESS_TOKEN_SECRET")

In [6]:
tweepy.__version__

'4.10.1'

In [7]:
client = tweepy.Client( bearer_token=TWITTER_BEARER_TOKEN, 
                        consumer_key=TWITTER_API_KEY, 
                        consumer_secret=TWITTER_API_SECRET_KEY, 
                        access_token=TWITTER_ACCESS_TOKEN, 
                        access_token_secret=TWITTER_ACCESS_TOKEN_SECRET, 
                        return_type = requests.Response,
                        wait_on_rate_limit=True)

# Find recent tweets from Elon Musk:

Right now, API returns a max of 100 tweets over the past 7 days. API access needs to be upgraded from "essential" in order to go back more days. 
Other fields besides 'author_id' and 'created_at' can be returned as well.


In [8]:
#https://www.kirenz.com/post/2021-12-10-twitter-api-v2-tweepy-and-pandas-in-python/twitter-api-v2-tweepy-and-pandas-in-python/

# Define query
query = 'from:elonmusk'# -is:retweet'
# get max. 100 tweets
tweets = client.search_recent_tweets(query=query, 
                                    tweet_fields=['author_id', 'created_at'], #there are more fields we could grab
                                     max_results=100)

In [9]:
# Save data as dictionary
tweets_dict = tweets.json() 
# Extract "data" value from dictionary
tweets_data = tweets_dict['data'] 
# Transform to pandas Dataframe
df = pd.json_normalize(tweets_data) 
df.head(5)

,author_id,edit_history_tweet_ids,created_at,text,id
0,44196397,[1575581872913907712],2022-09-29T20:22:46.000Z,@MuskUniversity True,1575581872913907712
1,44196397,[1575509494510993408],2022-09-29T15:35:09.000Z,Needs be able to get from Starbase to South Pa...,1575509494510993408
2,44196397,[1575508878300618752],2022-09-29T15:32:42.000Z,@WholeMarsBlog Off-label use 🤣,1575508878300618752
3,44196397,[1575508498430820352],2022-09-29T15:31:12.000Z,Cybertruck will be waterproof enough to serve ...,1575508498430820352
4,44196397,[1575507735298207744],2022-09-29T15:28:10.000Z,@phibetakitten Submarines use electric motors ...,1575507735298207744


# Example model: buy Dogecoin on Tuesday, sell Dogecoin on Friday

In [10]:
#Determine the day of the week: 

#https://stackoverflow.com/questions/9847213/how-do-i-get-the-day-of-week-given-a-date
#monday is 0, sunday is 6
#buy on Tuesday (1) and sell on Friday (4)

action = "none"

day_of_the_week = datetime.today().weekday()
if (day_of_the_week == 1):
    print("buy dogecoin")
    action = "buy"
elif (day_of_the_week == 4):
    print("sell dogecoin - make sure you have dogecoin in your account to sell")
    action = "sell"
else:
    print("take no action, it's not Tuesday or Friday")
    action = "none"

take no action, it's not Tuesday or Friday


In [11]:
# Check Dogecoin's latest price

#Dogecoin historical data collection

#https://alpaca.markets/learn/code-cryptocurrency-live-trading-bot-python-alpaca/
doge_data = api.get_crypto_bars("DOGEUSD", TimeFrame.Hour).df
doge_data.tail()
#exhanges are CBSE, ERSX, FTXU

,exchange,open,high,low,close,volume,trade_count,vwap
timestamp,,,,,,,,
2022-09-29 18:00:00+00:00,FTXU,0.059863,0.059992,0.059788,0.059791,64742,19,0.059908
2022-09-29 19:00:00+00:00,FTXU,0.059854,0.070528,0.056594,0.060081,15602134,289,0.060482
2022-09-29 20:00:00+00:00,FTXU,0.060075,0.060436,0.060062,0.060436,3750255,167,0.060146
2022-09-29 21:00:00+00:00,FTXU,0.060447,0.060598,0.060333,0.060333,480483,69,0.060535
2022-09-29 22:00:00+00:00,FTXU,0.060348,0.060637,0.060254,0.060415,533444,65,0.060428


In [12]:
# Check your current position in Dogecoin

#Get dogecoin position
#If this statement errors out, your account currently doesn't have any dogecoin in it

#https://alpaca.markets/learn/code-cryptocurrency-live-trading-bot-python-alpaca/
# Get current position on Bitcoin. Yields error if we don't hold any
dogecoin_qty_to_sell = api.get_position('DOGEUSD').qty
print(f"your paper trading accout currently has {dogecoin_qty_to_sell} dogecoins")
if (float(dogecoin_qty_to_sell) < 1 ) & (float(dogecoin_qty_to_sell) > 0 ):
    print("\nyou have less than 1 dogecoin, you won't be able to sell this through the API because the size is too small. Increase the amount of dogecoin you have to > 1 in order to place a sell order without error.")

your paper trading accout currently has 998.49 dogecoins


In [13]:
action

'none'

In [14]:
# Place trades with Alpaca API 

if action == "buy":
    print("place buy order with alpaca api")
    api.submit_order('DOGEUSD', qty=1000, side='buy', time_in_force="gtc")
elif action == "sell":
    print("place sell order with alpaca api")
    api.submit_order('DOGEUSD', qty=dogecoin_qty_to_sell, side='sell', time_in_force="gtc")
else:
    print("take no action")

take no action


In [15]:
# list your open orders to make sure that everything has closed: 

# list all of your open orders: 

#https://forum.alpaca.markets/t/cancel-all-the-open-orders-using-python-api/2890/2

open_orders_list = api.list_orders(status='open')
open_orders_list

[]

In [16]:
# check your dogecoin position

#Get dogecoin position
#If this statement errors out, your account currently doesn't have any dogecoin in it

#https://alpaca.markets/learn/code-cryptocurrency-live-trading-bot-python-alpaca/
# Get current position on Bitcoin. Yields error if we don't hold any
dogecoin_qty_to_sell = api.get_position('DOGEUSD').qty
print(f"your paper trading accout currently has {dogecoin_qty_to_sell} dogecoins")

your paper trading accout currently has 998.49 dogecoins


In [ ]:
# print closed orders

closed_orders = api.list_orders(
    status='closed',
    limit=100,
    nested=True  # show nested multi-leg orders
)

closed_orders = [o for o in closed_orders if o.symbol == 'TSLA']
print(closed_orders)

# Example model: buy Dogecoin if Elon tweets about it

In [17]:
# Check for Doge mentions
df["upper"] = df["text"].str.upper()
df_doge = df.loc[df["upper"].str.contains("DOGE")].copy()
print(df_doge)
print(f"Elon Musk has {len(df_doge)} recent tweets about Dogecoin")

Empty DataFrame
Columns: [author_id, edit_history_tweet_ids, created_at, text, id, upper]
Index: []
Elon Musk has 0 recent tweets about Dogecoin


In [18]:
if len(df_doge) > 0:
    print("recent tweets, buying dogecoin")
    api.submit_order('DOGEUSD', qty=1000, side='buy', time_in_force="gtc")
    print("you may want to double check your account details to make sure the trade closed successfully")
else:
    print("no recent tweets, no actions")

no recent tweets, no actions


# Example model: buy Bitcoin if Elon tweets about it

In [19]:
# Check for bitcoin mentions
searchfor = ["BITCOIN", "BTC"]
df_btc = df[df["upper"].str.contains('|'.join(searchfor))].copy()
print(df_btc)
print(f"Elon Musk has {len(df_btc)} recent tweets about Bitcoin")

Empty DataFrame
Columns: [author_id, edit_history_tweet_ids, created_at, text, id, upper]
Index: []
Elon Musk has 0 recent tweets about Bitcoin


In [20]:
if len(df_btc) > 0:
    print("recent tweets, buying bitcoin")
    api.submit_order('BTCUSD', qty=1000, side='buy', time_in_force="gtc")
    print("you may want to double check your account details to make sure the trade closed successfully")
else:
    print("no recent tweets, no actions")

no recent tweets, no actions


# Example model: buy Dogecoin if Elon has tweeted about it lately and there's postive sentiment on average

Sentiment can be defined in different ways, this example uses an average threshold of 0.5 to indicate positive sentiment

NO ACTIONS if on recent tweets about Dogecoin

In [21]:
if len(df_doge)> 0:
    #add sentimement analysis
    print("Creating sentiment scores")
    sid = SentimentIntensityAnalyzer()
    df_doge[['neg', 'neu', 'pos', 'compound']] = df_doge['text'].apply(sid.polarity_scores).apply(pd.Series)
    avg_sent = df_doge["compound"].mean()
    print(f"average sentiment score is {avg_sent}")
    if avg_sent > 0.5:
        api.submit_order('BTCUSD', qty=1000, side='buy', time_in_force="gtc")
        print("you may want to double check your account details to make sure the trade closed successfully")
    else:
        print("sentiment isn't high enough to buy yet, threshold was 0.5 so no actions taken")
        
else:
    print("no actions, no recent tweets")

no actions, no recent tweets


# Example models: SVM / Dogecoin recommendations

In [22]:
# Pull historical data
doge_data = api.get_crypto_bars("DOGEUSD", TimeFrame.Day, "2021-01-12", "2022-09-29").df
doge_data["Actual Returns"] = doge_data["close"].pct_change() #pct change
doge_data = doge_data.dropna()

# Add SMA calculations
short_window = 5
long_window = 30
doge_data['SMA_Fast'] = doge_data['close'].rolling(window=short_window).mean()
doge_data['SMA_Slow'] = doge_data['close'].rolling(window=long_window).mean()
doge_data = doge_data.dropna()

# Add signal columns
# Initialize the new Signal column
doge_data['Signal'] = 0
# When Actual Returns are greater than or equal to 0, generate signal to buy stock long
doge_data.loc[(doge_data['Actual Returns'] >=0), 'Signal'] = 1
# TRANSLATION: IF THE DAILY PRICE CHANGE WAS POSITIVE, THEN BUY LONG. IF IT WAS NEGATIE, THEN SELL STOCK SHORT
# TRANSLATION: SINCE WE CAN'T SHORT CRYPTO, THE (-1) SIGNAL WILL BE THE SIGNAL TO CLOSE OUR POSITION
# When Actual Returns are less than 0, generate signal to sell stock short
doge_data.loc[(doge_data['Actual Returns'] < 0), 'Signal'] = -1

# Incorporate the models 
X = doge_data[['SMA_Fast', 'SMA_Slow']].shift().dropna()
y = doge_data['Signal']

# Select the start of the training period
training_begin = X.index[0]#.index.min()

# Select the end of the training period
from pandas.tseries.offsets import DateOffset
training_end = X.index[0] + DateOffset(months=13)

# Generate the X_train and y_train DataFrames
X_train = X.loc[training_begin:training_end]
y_train = y.loc[training_begin:training_end]

# Generate the X_test and y_test DataFrames
X_test = X.loc[training_end+DateOffset(hours=24):]
y_test = y.loc[training_end+DateOffset(hours=24):]

# Scale the features DataFrames
from sklearn.preprocessing import StandardScaler

# Apply the scaler model to fit the X-train data
scaler = StandardScaler()
X_scaler = scaler.fit(X_train)

# Transform the X_train and X_test DataFrames using the X_scaler
X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)

# Create the svm model
# Create the svm classifier model & create predictions dataset
from sklearn import svm
svm_model = svm.SVC()
# Use the testing data to make the model predictions
svm_model = svm_model.fit(X_train_scaled, y_train)
svm_pred = svm_model.predict(X_test_scaled)
if svm_pred[len(svm_pred)-1] == 1:
    print("SVM dogecoin model recommends to BUY dogecoin")
elif svm_pred[len(svm_pred)-1] == -1:
    print("SVM dogeoin model recommends to SELL dogecoin")
else:
    print("Other condition, check SVM dogecoin model")

SVM dogecoin model recommends to BUY dogecoin


# Other Account Actions

## Place trades with Alpaca trading API

As of the time of writing the quantity of 1000 dogecoins corresponds to approx $60 USD worth

In [ ]:
if buy_dogecoin == True:
    api.submit_order('DOGEUSD', qty=1000, side='buy', time_in_force="gtc")

## Make sure your order when through (open orders list should be empty)

In [22]:
# list all of your open orders: 

#https://forum.alpaca.markets/t/cancel-all-the-open-orders-using-python-api/2890/2

open_orders_list = api.list_orders(status='open')
open_orders_list

[]

## Get details about the order that you recently placed

In [ ]:
# print closed orders

closed_orders = api.list_orders(
    status='closed',
    limit=100,
    nested=True  # show nested multi-leg orders
)

closed_orders = [o for o in closed_orders if o.symbol == 'DOGEUSD']
print(closed_orders)